In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import datetime

In [2]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
import tensorflow as tf
from tensorflow.keras.layers import Conv1D, LSTM, Dense, Dropout
from tensorflow.keras.layers import MaxPooling1D, Flatten
from tensorflow.keras.regularizers import L1, L2
from tensorflow.keras.metrics import Accuracy
from tensorflow.keras.metrics import RootMeanSquaredError

In [3]:
def import_et_scale():
    global X_train, y_train, dataset_train, sc
    dataset_train = pd.read_csv('Google_Stock_Price_Train.csv')
    training_set = dataset_train.iloc[:, 1:2].values
    
    sc = MinMaxScaler(feature_range = (0, 1))
    training_set_scaled = sc.fit_transform(training_set)
    
    X_train = []
    y_train = []
    for i in range(60, 1258):
        X_train.append(training_set_scaled[i-60:i, 0])
        y_train.append(training_set_scaled[i, 0])
    X_train, y_train = np.array(X_train), np.array(y_train)
    
    X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))
    print(X_train.shape)

In [4]:
def LSTM_first_layer():
    global dropout_LSTM_layer, X_train, regressor
    print('Combien d\'units mettre dans cette couche ? (50 de base)')
    LSTM_units = int(input())
    if LSTM_units == '':
        LSTM_units = 50
    regressor.add(LSTM(units=LSTM_units, return_sequences=True, input_shape=(X_train.shape[1],1)))
    regressor.add(Dropout(dropout_LSTM_layer))

In [5]:
def LSTM_layer():
    global LSTM_layers, dropout_LSTM_layer, regressor
    print('Combien d\'units mettre dans cette couche ? (50 de base)')
    LSTM_units = int(input())
    if LSTM_units == '':
        LSTM_units = 50
    regressor.add(LSTM(units=LSTM_units, return_sequences=True))
    regressor.add(Dropout(dropout_LSTM_layer))

In [12]:
def model():
    #Variables global
    global regressor, hist, X_train, y_train, dropout_LSTM_layer, LSTM_layers, dataset_train, sc, n_epochs, ze_batch_size

    regressor = Sequential()
    print('Entrer le nombre de couches LSTM que vous voulez')
    LSTM_layers = int(input())
    print('Choisissez le taux de Dropout pour ces couches (0.1, 0.15, 0.2...)')
    dropout_LSTM_layer = float(input())
    if LSTM_layers > 1:
        LSTM_first_layer()
        for LSTM_layers in range((LSTM_layers-1)):
            LSTM_layer()
    else:
        LSTM_first_layer()

    # The output layer
    regressor.add(Dense(units=1))
    
    # Compiling the RNN
    regressor.compile(optimizer='adam',loss='mean_squared_error')
    
    print('Choisissez le nombre d\'epoch')
    n_epochs = int(input())
    print('Choisissez le batch_size')
    ze_batch_size = int(input())
    # Fitting to the training set
    hist = regressor.fit(X_train,y_train,epochs=n_epochs,batch_size=ze_batch_size)
    return hist

In [13]:
def real_stock_price():
    global regressor, hist, X_train, y_train, dropout_LSTM_layer, LSTM_layers, dataset_train
    
    dataset_test = pd.read_csv('Google_Stock_Price_Test.csv')
    real_stock_price = dataset_test.iloc[:, 1:2].values
    dataset_total = pd.concat((dataset_train['Open'], dataset_test['Open']), axis = 0)
    inputs = dataset_total[len(dataset_total) - len(dataset_test) - 60:].values
    inputs = inputs.reshape(-1,1)
    inputs = sc.transform(inputs)
    X_test = []
    for i in range(60, 80):
        X_test.append(inputs[i-60:i, 0])
    X_test = np.array(X_test)
    print(X_test.shape)
    X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))
    print(X_test.shape)
    predicted_stock_price = hist.predict(X_test)
    print(predicted_stock_price)
    print(predicted_stock_price.shape)
    #predicted_stock_price = sc.inverse_transform(predicted_stock_price)

In [8]:
import_et_scale()

(1198, 60, 1)


In [9]:
model()

Entrer le nombre de couches LSTM que vous voulez
1
Choisissez le taux de Dropout pour ces couches (0.1, 0.15, 0.2...)
0.2
Combien d'units mettre dans cette couche ? (50 de base)
1
Choisissez le nombre d'epoch
1
Choisissez le batch_size
32
38/38 [==============================] - 5s 26ms/step - loss: 0.7828


In [10]:


print(X_train[0])
print(y_train[0])

[[0.08581368]
 [0.09701243]
 [0.09433366]
 [0.09156187]
 [0.07984225]
 [0.0643277 ]
 [0.0585423 ]
 [0.06568569]
 [0.06109085]
 [0.06639259]
 [0.0614257 ]
 [0.07474514]
 [0.02797827]
 [0.02379269]
 [0.02409033]
 [0.0159238 ]
 [0.01078949]
 [0.00967334]
 [0.01642607]
 [0.02100231]
 [0.02280676]
 [0.02273235]
 [0.02810849]
 [0.03212665]
 [0.0433812 ]
 [0.04475779]
 [0.04790163]
 [0.0440695 ]
 [0.04648783]
 [0.04745517]
 [0.04873875]
 [0.03936305]
 [0.04137213]
 [0.04034898]
 [0.04784582]
 [0.04325099]
 [0.04356723]
 [0.04286033]
 [0.04602277]
 [0.05398467]
 [0.05738894]
 [0.05714711]
 [0.05569611]
 [0.04421832]
 [0.04514845]
 [0.04605997]
 [0.04412531]
 [0.03675869]
 [0.04486941]
 [0.05065481]
 [0.05214302]
 [0.05612397]
 [0.05818885]
 [0.06540665]
 [0.06882953]
 [0.07243843]
 [0.07993526]
 [0.07846566]
 [0.08034452]
 [0.08497656]]
0.08627874097775134


In [11]:
real_stock_price()

(20, 60)
(20, 60, 1)


NameError: name 'hist' is not defined